In [2]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR + 'code/CNN/cnn_ext_model/ext_model.ipynb'

In [3]:
%run {RUN_DIR}

In [5]:
class RnnBasicModel(CnnExtModel):
    pass

In [10]:
def rnn_basic_alloc_rnn_layer(self, input_shape, hconfig):
    # 입력 출력의 시계열 인지 확인
    #input_shape = > 40,43 [timesteps1, timefeats]
    inseq = get_conf_param(hconfig, 'inseq', True)
    outseq = get_conf_param(hconfig, 'outseq', True)
    target_num = 2 # target
    if inseq:
    # 시간대최대 길이timesteps1, 시간대별 입력 벡터 크기= timefeats
    # 시간축 크기를 나타내며 길이표시를 위해
    # 한칸더 할당되었음을 표시하기 위해 1을 붙임
    #timefeats -> x1~xn
    
        timesteps1, timefeats = input_shape
    else:
        timesteps1 = get_conf_param(hconfig, 'timesteps') + 1
        timefeats = np.prod(input_shape)
    # 순환백터의 크기     
    # hidden
    recur_size = get_conf_param(hconfig, 'recur_size')
    # 가중치와 편향파라미터 생성
    ex_inp_dim = timefeats + recur_size
    # 시간대별입력 + 순환벡터의 확장크기 => w_param
    # bias -> recur_size
#     weight, bias = self.alloc_param_pair([ex_inp_dim, recur_size])
    # timesteps 1 -> 나중에 forward or back 반복제어용 
    if outseq:
        # 시계열데이터일경우
        output_shape = [timesteps1, recur_size]
        
    else:
        # 시계열 데이터가 아닐경우 
        
        output_shape = [recur_size]
    
    rnn = RNN(timefeats, recur_size, target_num, 30)
    return [rnn], output_shape

#     return {'w':weight, 'b': bias, 'info':rnn_info}, output_shape

RnnBasicModel.alloc_rnn_layer = rnn_basic_alloc_rnn_layer

SyntaxError: positional argument follows keyword argument (Temp/ipykernel_5172/1631570774.py, line 34)

In [8]:
import torch.nn as nn

In [7]:
class RNN(nn.Module):
    def __init__(self,input_size, hidden_size, output_size, batch_size):
        super(RNN, self).__init__()
        
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.output_size = 2
        self.n_layers= 2
        
        ## layers
        
        self.rnn = nn.RNN(input_size = self.input_size, hidden_size =self.hidden_size,\
                          num_layers =self.n_layers, batch_first=True,\
                         dropout=0.9)
#         , nonlinearity='relu'
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
#         self.i2o = nn.Linear(self.input_size + self.hidden_size, self.output_size)
#         self.i2h = nn.Linear(self.input_size+self.hidden_size, self.hidden_size)
    def forward(self, x):
        x = x[:,1:,:]
#         print(x.shape)
        h_0 = self._init_state(batch_size = x.size(0))
#         print(h_0.shape,type(h_0))
        out, h = self.rnn(x, h_0)
#         print(out.shape)
        h_t = out[:,-1,:]
#         print(h_t.shape)
        logit = self.out(h_t)
        
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_size).zero_()

In [ ]:
def eval_accuracy_torch(self, x, y, output=None):
    if output is None:
        with torch.no_grad():
            output = self.model.forward(x)

    accuracy = self.dataset.eval_accuracy(x, y, output)
    
    return accuracy

RnnBasicModel.eval_accuracy = eval_accuracy_torch

In [ ]:
def test_torch(self,batch_size):
    with torch.no_grad():
#         print(self.model.parameters)
        test_X,test_y=self.dataset.get_test_data(batch_size)
        test_X=torch.from_numpy(test_X).float()
        test_y=torch.from_numpy(test_y).float()
#         print(test_X.shape,test_y.shape,'테스트')
        time1 = int(time.time())        
        acc = self.eval_accuracy(test_X, test_y)
        time2 = int(time.time())
        self.dataset.test_prt_result(self.name, acc, time2-time1)

RnnBasicModel.test = test_torch

In [ ]:
def mlp_get_estimate_torch(self, x):
    with torch.no_grad():
#         x=x.view([-1,3,96,96])
#         x=torch.from_numpy(x).float()
        output = self.model.forward(x)
    estimate = self.dataset.get_estimate(output)
    return estimate

RnnBasicModel.get_estimate = mlp_get_estimate_torch

In [ ]:
def mlp_model_visualize_torch(self, num):
    print('Model {} Visualization'.format(self.name))
    deX, deY = self.dataset.get_visualize_data(num)
    deX_=torch.from_numpy(deX).float()
    deY_=torch.from_numpy(deY).float()
    est = self.get_estimate(deX_)
    self.dataset.visualize(deX, est, deY)

RnnBasicModel.visualize = mlp_model_visualize_torch

In [ ]:
def init_parameters_torch(self, hconfigs):
    
    self.hconfig = hconfigs
    
    prev_shape = self.dataset.input_shape
    for hconfig in hconfigs:
        pm, prev_shape=self.alloc_layer_param(prev_shape, hconfig)
        self.layers.append(pm)
        
    output_cnt = int(np.prod(self.dataset.output_shape))
#     self.layers.append([nn.Flatten()])
#     pm, _ =self.alloc_layer_param(prev_shape, output_cnt)
    
#     self.layers.append(pm)
    self.layers=sum(self.layers,[])
RnnBasicModel.init_parameters = init_parameters_torch


In [9]:
import torch

In [ ]:
def rnn_eval_accuracy_torch(self, x, y, output=None):
    if output is None:
        with torch.no_grad():
#             x=x.view([-1,3,96,96])
            if isinstance(x, torch.Tensor):
                pass
            else:
                x=torch.from_numpy(x).float()
                y=torch.from_numpy(y).float()
            output = self.model.forward(x)
    accuracy = self.dataset.eval_accuracy(x, y, output)
    
    return accuracy

RnnBasicModel.eval_accuracy = rnn_eval_accuracy_torch